In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import random

In [1]:
def getdata():
    train = pd.read_csv('pa3_train.csv')
    train.shape
    val = pd.read_csv('pa3_val.csv')
    test = pd.read_csv('pa3_test.csv')
    #print(train.shape[1])
    del train['veil-type_p']
    del test['veil-type_p']
    del val['veil-type_p']
    #print(train.shape[1])
    y_train = np.matrix(train.loc[:,['class']])
    x_train = np.matrix(train.drop(['class'],axis=1))
    y_val = np.matrix(val.loc[:,['class']])
    x_val = np.matrix(val.drop(['class'],axis=1))
    x_test = np.matrix(test)
    
    return x_train,y_train,x_val,y_val,x_test

class DTree:
    def __init__(self,feature_index,leaf,propy,depth):
        self.feature_index = feature_index
        self.leaf = leaf
        self.propy = propy
        self.depth = depth   
    def leftnode(self, node):
        self.leftnode = node
    def rightnode(self, node):
        self.rightnode = node
        
def gini(y):
    c0 = 0
    c1 = 0
    for i in y:
        if (i == 0):
            c0 += 1
        else:
            c1 += 1
    t = c0 + c1       
    neg = c0 / t
    pos = c1 / t    
    g = 1 - (neg*neg) - (pos*pos)
    return g
   
def benefit(x,feature,y):
    u =0
    gini_imp = []
    benefit=0
    c= x[:,feature].tolist()
    vals,count = np.unique(c,return_counts=True)
    vcount = len(vals)
    total = np.sum(count)
    # Calculating U(Y)
    u = gini(y)
    for item in vals:
        feat = x.copy()
        index = np.array(np.where(feat[:,feature] == item))[0]
        y_sub = y[index]
        gini_imp.append(gini(y_sub))
    if (len(gini_imp)==2):
        benefit = u- (gini_imp[0]*(count[0]/total)) - (gini_imp[1]*(count[1]/total))
    else:
        benefit = u- (gini_imp[0]*(count[0]/total))
    #print(benefit)
    return benefit
                       

def select_gini(x_train,y_train):
   
    atr_list = []
    for i in range(0,x_train.shape[1]):
         atr_list.append(i)
    gini_weights = [0]*len(atr_list)
   
    for item in atr_list:
        gini_weights[item] = benefit(x_train,item,y_train)   
            
    index_max = np.argmax(gini_weights)
    return gini_weights[index_max], atr_list[index_max]

def make_tree(x_train,y_train,propy,depth,max_depth):
    #print("Number of times code is run index")
     if (depth <= max_depth):
        if (depth==max_depth) or (propy==1) or (propy==0):
            node = DTree(-2,1,propy,depth)
            return node
        
        gini_weights,feature_index = select_gini(x_train,y_train)
        #print(gini_weights)
        #print(feature_index)
        X_left = []
        Y_left = []
        X_right = []
        Y_right = []
        n = x_train.shape[0]
        m =x_train.shape[1]
        #print("Making Splits")
        for i in  range(0,n):
            c= x_train[:,feature_index].tolist()
            vals,count = np.unique(c,return_counts=True)
            if (vals[0]!=0):
                print("Exception Error")
            else:
                if (x_train[i,feature_index].item() == vals[0]):
                    if (len(X_left) == 0):
                        X_left = x_train[i]
                        Y_left = y_train[i]
                    else :
                        X_left = np.vstack((X_left, x_train[i]))
                        Y_left = np.vstack((Y_left, y_train[i]))
                else :
                    if (len(X_right) == 0):
                        X_right = x_train[i]
                        Y_right = y_train[i]
                    else:
                        X_right = np.vstack((X_right, x_train[i]))
                        Y_right = np.vstack((Y_right, y_train[i]))
        #print("Spliting completed")
        node = DTree(feature_index,0,propy, depth)
        propy_left = cal_propy(Y_left)
        propy_right = cal_propy(Y_right)
        if (len(Y_left) == 0):
            print("no examples")
        else:
            #print("Attaching another node at left ")
            node.leftnode(make_tree(X_left, Y_left,propy_left,depth+1, max_depth))
            #print("Attached")

        if (len(Y_right) == 0):
            print("no examples")
        else:
            #print("Attaching another node at right ")
            node.rightnode(make_tree(X_right, Y_right,propy_right,depth+1, max_depth))
            #print("Attached")
        print("done")
        return node
    
def cal_propy(Y):
    return (np.sum(Y)/len(Y))
def calculate_accuracy(x,y,node):
    
    if (node.leaf==1):
        if (node.propy > 0.5 and y == 1) or (node.propy < 0.5 and y == 0):
            return True
        else:
            return False

    if (node.feature_index == -3):
        return True
    else :   
        if (x[0,node.feature_index] == 0) :
            return calculate_accuracy(x,y,node.leftnode)
        else:
            return calculate_accuracy(x,y,node.rightnode)

def accuracy(x_train, y_train, root, max_depth):
    correct_pred = 0
    for i in range(x_train.shape[0]):
        if calculate_accuracy(x_train[i], y_train[i], root):
            correct_pred += 1
    accuracy = (correct_pred*100)/x_train.shape[0]
    return accuracy

def Part1(x_train,y_train,x_val,y_val):
    print("Starting Part 1")
    propy = cal_propy(y_train)
    max_depth=8
    accuracy_train = []
    accuracy_val = []
    a=0
    b=0
   # Part a)
    
    root = make_tree(x_train,y_train,propy,0, 2)
    
    
    
    #Part b)
    a = accuracy(x_train,y_train,root,max_depth)
    b = accuracy(x_val,y_val,root,max_depth)
    print("Train Accuracy for Part a",a)
    print("Val Accuracy for Part a",b)
    
    
    #Part c)
    print("making trees for diffent depth")
    d=[]
    for i in range(1,9):
        d.append(i)
        print("D:",d)
        writer = csv.writer(open("Part1training"+str(i)+".csv","w",newline=''), delimiter=',',quoting=csv.QUOTE_ALL)
        root = make_tree(x_train,y_train,propy,0,i)
        a = accuracy(x_train,y_train,root,i)
        b = accuracy(x_val,y_val,root,i)
        writer.writerow(("Accuracy training =","depth",a,i))
        writer.writerow(("Accuracy validation =","depth",b,i))
        accuracy_train.append(a)
        accuracy_val.append(b)
    print("Done with Part 1)")
    plt.figure()
    plt.plot(d, accuracy_train, label = "Training Accuracy",color='blue', linestyle='solid', linewidth = 1.5 )
    plt.plot(d, accuracy_val, label = "Validation Accuracy",color='orange', linestyle='solid', linewidth = 1.5 )
    plt.xlabel('Depth of Tree', fontsize=12)
    plt.ylabel('Accuracies \n max_validation_accuracy=' + str(np.max(accuracy_val)) + " at depth=" + str(np.argmax(accuracy_val)) + '\n max_training_accuracy=' + str(np.max(accuracy_train)) + " at depth=" + str(np.argmax(accuracy_train))  , fontsize=11)
    plt.title('Accuracies vs Depth of Decision Tree', fontsize=11)
    plt.legend()
    plt.grid()
    x1,x2,y1,y2 = plt.axis()
    plt.savefig("Part1_Accuracies_vs_Depth.png", dpi = 300,bbox_inches="tight")
    plt.show()

def random_forest(x_train,y_train,x_val,y_val,tree_num,feat_num,max_depth,seed=0):
    root_nodes = np.zeros((tree_num,),dtype=DTree)
    accuracy_train =0
    accuracy_val = 0
    for i in range(0,tree_num):
        np.random.seed(seed)
        index = np.random.randint(x_train.shape[0], size=x_train.shape[0])
        x_sampled = x_train[index]
        y_sampled = y_train[index]
        propy = cal_propy(y_sampled)
        root_nodes[i] = make_tree2(x_sampled,y_sampled,propy,feat_num,0,max_depth)
    accuracy_train = accuracy2(x_train, y_train, root_nodes, max_depth)
    accuracy_val = accuracy2(x_val, y_val, root_nodes, max_depth)
    return accuracy_train,accuracy_val


def calculate_accuracy2(x,node):
    if (node.leaf == 1):
        if (node.propy > 0.5):
            return 1
        else:
            return 0
    if (x[0,node.feature_index] == 0):
        return calculate_accuracy2(x,node.leftnode)
    else:
        return calculate_accuracy2(x,node.rightnode)

def accuracy2(X, Y, root_nodes, max_depth):
    correct_pred = 0
    for i in range(X.shape[0]):
        majority_prediction = np.zeros(len(root_nodes))
        #print(i)
        for j in range(len(root_nodes)):
            #print(j)
            majority_prediction[j] = calculate_accuracy2(X[i],root_nodes[j])
        #print("MP",majority_prediction)
        prop = np.sum(majority_prediction)/len(root_nodes)
        #print("prop",prop)
        if (prop > 0.5 and Y[i] ==1 ) or (prop < 0.5 and Y[i] ==0 ) :
            correct_pred += 1
    accuracy = (correct_pred*100)/X.shape[0]
    return accuracy

def select_gini2(x_train,y_train,feat_num,seed=0):
     
    atr_list = []
    np.random.seed(seed)
    sampled_index = random.sample(range(x_train.shape[1]),feat_num)
    x_train_sub = x_train[:,sampled_index]
    for i in range(0,x_train_sub.shape[1]):
         atr_list.append(i) 
    gini_weights = np.zeros(x_train_sub.shape[1])
    for item in range(0,x_train_sub.shape[1]):
        #print("f",item)
        gini_weights[item] = benefit(x_train_sub,item,y_train)   
            
    index_max = np.argmax(gini_weights)
    return gini_weights[index_max], sampled_index[index_max]


def make_tree2(x_train,y_train,propy,feat_num,depth,max_depth,seed=0):
    if (depth <= max_depth):
        
        if (depth==max_depth) or (propy==1) or (propy==0) :
            #print("Making leaf node")
            node = DTree(-2,1,propy,depth)
            return node
        
        gini_weights,feature_index = select_gini2(x_train,y_train,feat_num,seed)
        #print(gini_weights)
        X_left = []
        Y_left = []
        X_right = []
        Y_right = []
        n = x_train.shape[0]
        m =x_train.shape[1]
        #print("Making Splits")  
        for i in  range(0,n):
                if (x_train[i,feature_index].item() == 0):
                    if (len(X_left) == 0):
                        X_left = x_train[i]
                        Y_left = y_train[i]
                    else :
                        X_left = np.vstack((X_left, x_train[i]))
                        Y_left = np.vstack((Y_left, y_train[i]))
                else :
                    if (len(X_right) == 0):
                        X_right = x_train[i]
                        Y_right = y_train[i]
                    else:
                        X_right = np.vstack((X_right, x_train[i]))
                        Y_right = np.vstack((Y_right, y_train[i]))
        Y_left = np.matrix(Y_left)
        Y_right = np.matrix(Y_right)                                    
        node = DTree(feature_index,0,propy, depth)
        propy_left = cal_propy(Y_left)
        propy_right = cal_propy(Y_right)
        if (len(Y_left) == 0):
             print("no examples")
        else:
            #print("Attaching another node at left ")
            node.leftnode(make_tree2(X_left, Y_left,propy_left,feat_num,depth+1, max_depth))
            #print("Attached")

        if (len(Y_right) == 0):
             print("no examples")
        else:
            #print("Attaching another node at right ")
            node.rightnode(make_tree2(X_right, Y_right,propy_right,feat_num,depth+1, max_depth))
            #print("Attached")
                #print("done")
        return node


def Part2(x_train,y_train,x_val,y_val):
    print("Starting Part 2)")
    #Part a)
    print("Random Forest for different number of trees")
    max_depth=2
    feat_num=5
    tree_num = [1,2,5,10,25]
    accuracy_train = np.zeros(len(tree_num))
    accuracy_val = np.zeros(len(tree_num))
    
    for i in range(0,len(tree_num)):
        print("No.of.Trees:", tree_num[i])
        writer = csv.writer(open("Part2training"+str(tree_num[i])+".csv","w",newline=''), delimiter=',',quoting=csv.QUOTE_ALL)
        accuracy_train[i], accuracy_val[i]  = random_forest(x_train,y_train,x_val,y_val,tree_num[i],feat_num,max_depth,0)
        writer.writerow(("Accuracy training =","i",accuracy_train[i],tree_num[i]))
        writer.writerow(("Accuracy validation =","i",accuracy_val[i],tree_num[i]))
    print("Done")
    plt.figure()
    plt.plot(tree_num,accuracy_train, label='Training Accuracy', color='blue', linewidth=1.5,marker=".")
    plt.plot(tree_num,accuracy_val, label='Validation Accuracy', color='orange', linewidth=1.5,marker=".")
    plt.title('Training and Validation Accuracy vs No. of trees ' )
    plt.legend(loc='best')
    plt.xlabel(  ("No. of Trees \nMax Depth(d) ="+str(max_depth)+', No. of features selected(m)='+ str(feat_num) ))
    plt.ylabel('Accuracy \nMax Training: '+str(np.max(accuracy_train))+' \n Max Validation: '+str(np.max(accuracy_val)))
    plt.grid()
    plt.savefig("Part2_AccuracyVsno_of_trees_depth_m_"+str(feat_num)+".png", dpi = 300,bbox_inches="tight")
    plt.show() 
    
    #Part d
    print("Random Forest for different number of features")
    max_depth=2
    tree_num = 15
    feat_num = [1, 2, 5, 10, 25, 50]
    accuracy_train = np.zeros(len(feat_num))
    accuracy_val = np.zeros(len(feat_num))
    for i in range(0,len(feat_num)):
        print("Number of Features:", feat_num[i])
        writer = csv.writer(open("Part2trainingPartd"+str(feat_num[i])+".csv","w",newline=''), delimiter=',',quoting=csv.QUOTE_ALL)
        accuracy_train[i], accuracy_val[i]  = random_forest(x_train,y_train,x_val,y_val,tree_num,feat_num[i],max_depth,0)
        writer.writerow(("Accuracy training =","i",accuracy_train[i],feat_num[i]))
        writer.writerow(("Accuracy validation =","i",accuracy_val[i],feat_num[i]))
    print("Done")
    plt.figure()
    plt.plot(feat_num,accuracy_train, label='Training Accuracy', color='blue', linewidth=1.5,marker=".")
    plt.plot(feat_num,accuracy_val, label='Validation Accuracy', color='orange', linewidth=1.5,marker=".")
    plt.title(' Training and Validation Accuracy vs No. of features ' )
    plt.legend(loc='best')
    plt.xlabel(  ("No. of Features \nMax Depth(d) ="+str(max_depth)+', No. of features selected(m)='+ str(feat_num) ))
    plt.ylabel('Accuracy \nMax Training: '+str(np.max(accuracy_train))+' \n Max Validation: '+str(np.max(accuracy_val)))
    plt.grid()
    plt.savefig("Part2_AccuracyVsNumber of feature"+str(feat_num)+".png", dpi = 300,bbox_inches="tight")
    plt.show() 
    
    
    # Part e)
    print("Random Forest for different seeds")
    random_seed = [11,66,128,444,712,981,1045,1760,3800,4547]
    max_depth = 2
    tree_num = 25
    feat_num = 25
    accuracy_train = np.zeros(len(random_seed))
    accuracy_val = np.zeros(len(random_seed))
    for i in range(0,len(random_seed)):
        np.random.seed(random_seed[i])
        print("For Random Seed:",random_seed[i])
        writer = csv.writer(open("Part2trainingParte"+str(random_seed[i])+".csv","w",newline=''), delimiter=',',quoting=csv.QUOTE_ALL)
        accuracy_train[i], accuracy_val[i]  = random_forest(x_train,y_train,x_val,y_val,tree_num,feat_num,max_depth,random_seed[i])
        writer.writerow(("Accuracy training =","i",accuracy_train[i],random_seed[i]))
        writer.writerow(("Accuracy validation =","i",accuracy_val[i],random_seed[i]))
    print("Finished Part 2)")
    print("Done")
    plt.figure()
    plt.plot(random_seed,accuracy_train, label='Training Accuracy', color='blue', linewidth=1.5,marker="o")
    plt.plot(random_seed,accuracy_val, label='Validation Accuracy', color='orange', linewidth=1.5,marker="o")
    plt.title('Accuracy at different random seed' )
    plt.legend(loc='best')
    plt.xlabel(  ("Random Seeds \nMax Depth(d) ="+str(max_depth)+', No. of features selected(m)='+ str(feat_num) ))
    plt.ylabel('Accuracy \nMax Training: '+str(np.max(accuracy_train))+' \n Max Validation: '+str(np.max(accuracy_val)))
    plt.grid()
    plt.savefig("Part2_AccuracyatRandomSeed.png", dpi = 300,bbox_inches="tight")
    plt.show()    

def main():
    print("Starting code")
    x_train,y_train,x_val,y_val,x_test = getdata()
    #Part1(x_train,y_train,x_val,y_val)
    Part2(x_train,y_train,x_val,y_val)
    print("Done")   

In [ ]:
main()